In [ ]:
!pip install flask flask-cors pandas openpyxl dash requests --quiet

from flask import Flask, jsonify, request
from flask_cors import CORS
import pandas as pd

app = Flask(__name__)
CORS(app) 

# Carregar os dados 
ouvidoria = "ouvidoria-aneel-2023.xlsx"
df_ouvidoria = pd.read_excel(ouvidoria)

# Rota principal da API
@app.route('/aneel', methods=['GET'])
def teste_aneel():
    """
    Endpoint que retorna informações da API TESTE ANEEL V.1
    """
    return jsonify({
        "api_name": "TESTE ANEEL",
        "version": "1.0",
        "description": "API de teste para a ANEEL, desenvolvida para testes de estrutura RESTful.",
        "status": "success"
    })

# Rota para consulta por município http://localhost:3000/municipio/1200401
@app.route('/municipio/<int:codigoMunicipio>', methods=['GET'])
def consulta_municipio(codigoMunicipio):
    """
    Endpoint para consultar informações sobre reclamações, elogios, etc., de um município específico.
    """
    municipio_data = df_ouvidoria[df_ouvidoria['CodigoMunicipio'] == codigoMunicipio]
    if municipio_data.empty:
        return jsonify({"error": "Município não encontrado"}), 404
    
    result = []
    for _, row in municipio_data.iterrows():
        result.append({
            "municipio": row['NomMunicipio'],
            "uf": row['SigUF'],
            "categoria": row['NomCategoria'],
            "subcategoria": row['NomSubCategoria'],
            "quantidade": row['NumQtdReclamacoesDia'],
            "situacao": row['DscSituacao']
        })
    return jsonify(result)

# Rota para consulta por agente http://localhost:3000/agente/Energisa
@app.route('/agente/<string:sigAgente>', methods=['GET'])
def consulta_agente(sigAgente):
    """
    Endpoint para consultar informações de um agente específico.
    """
    agente_data = df_ouvidoria[df_ouvidoria['SigAgente'].str.contains(sigAgente, case=False)]
    if agente_data.empty:
        return jsonify({"error": "Agente não encontrado"}), 404
    
    result = []
    for _, row in agente_data.iterrows():
        result.append({
            "agente": row['SigAgente'],
            "uf": row['SigUF'],
            "municipio": row['NomMunicipio'],
            "categoria": row['NomCategoria'],
            "subcategoria": row['NomSubCategoria'],
            "quantidade": row['NumQtdReclamacoesDia'],
            "situacao": row['DscSituacao']
        })
    return jsonify(result)

# Rota para consultar reclamações por categoria http://localhost:3000/reclamacoes?categoria=Informação&subcategoria=Atendimento
@app.route('/reclamacoes', methods=['GET'])
def consulta_reclamacoes():
    """
    Endpoint para filtrar reclamações por categoria e subcategoria.
    """
    categoria = request.args.get('categoria')
    subcategoria = request.args.get('subcategoria')
    
    filtro = df_ouvidoria
    if categoria:
        filtro = filtro[filtro['NomCategoria'].str.contains(categoria, case=False)]
    if subcategoria:
        filtro = filtro[filtro['NomSubCategoria'].str.contains(subcategoria, case=False)]
    
    result = []
    for _, row in filtro.iterrows():
        result.append({
            "municipio": row['NomMunicipio'],
            "uf": row['SigUF'],
            "categoria": row['NomCategoria'],
            "subcategoria": row['NomSubCategoria'],
            "quantidade": row['NumQtdReclamacoesDia'],
            "situacao": row['DscSituacao']
        })
    return jsonify(result)

from werkzeug.serving import run_simple

if __name__ == '__main__':
    run_simple('localhost', 3000, app, use_debugger=False, use_reloader=False)


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import requests

# Inicializando a aplicação Dash
app = dash.Dash(__name__)

# Layout do Dashboard
app.layout = html.Div(
    [
        html.H1("Dashboard de Reclamações da ANEEL", style={'textAlign': 'center', 'marginTop': '50px'}),
        
        # Campo de input para o código do município
        html.Div(
            [
                html.Label("Digite o código do município:", style={'fontSize': '18px'}),
                dcc.Input(id="municipio-input", type="number", placeholder="Código do município", debounce=True, style={'fontSize': '18px', 'padding': '10px'}),
                html.Button("Consultar Município", id="consultar-btn", n_clicks=0, style={'fontSize': '18px', 'marginLeft': '10px', 'padding': '10px', 'backgroundColor': '#4CAF50', 'color': 'white'})
            ],
            style={'display': 'flex', 'justifyContent': 'center', 'marginTop': '30px'}
        ),
        
        # Exibição dos dados do município
        html.Div(id="output-municipio", style={'textAlign': 'center', 'marginTop': '20px', 'fontSize': '16px'}),
        
        # Filtros de categoria e subcategoria
        html.Div(
            [
                html.Label("Categoria:", style={'fontSize': '18px', 'marginTop': '20px'}),
                dcc.Dropdown(
                    id="categoria-dropdown",
                    options=[
                        {"label": "Informação", "value": "Informação"},
                        {"label": "Atendimento", "value": "Atendimento"}
                    ],
                    placeholder="Escolha uma categoria",
                    multi=True,
                    style={'fontSize': '18px', 'width': '40%', 'margin': 'auto'}
                ),
                
                html.Label("Subcategoria:", style={'fontSize': '18px', 'marginTop': '20px'}),
                dcc.Dropdown(
                    id="subcategoria-dropdown",
                    options=[
                        {"label": "Atendimento", "value": "Atendimento"},
                        {"label": "Informação", "value": "Informação"}
                    ],
                    placeholder="Escolha uma subcategoria",
                    multi=True,
                    style={'fontSize': '18px', 'width': '40%', 'margin': 'auto'}
                ),
                
                html.Button("Consultar Reclamações", id="consultar-reclamacoes", n_clicks=0, style={'fontSize': '18px', 'marginTop': '20px', 'backgroundColor': '#2196F3', 'color': 'white'})
            ],
            style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'}
        ),
        
        # Exibição das reclamações filtradas
        html.Div(id="output-reclamacoes", style={'textAlign': 'center', 'marginTop': '20px', 'fontSize': '16px'})
    ]
)

# Função para consultar o município
@app.callback(
    Output("output-municipio", "children"),
    Input("municipio-input", "value")
)
def consultar_municipio(codigoMunicipio):
    if not codigoMunicipio:
        return "Informe o código do município para consulta."

    url = f"http://localhost:3000/municipio/{codigoMunicipio}"
    response = requests.get(url)
    
    if response.status_code == 404:
        return f"Município com código {codigoMunicipio} não encontrado."
    
    data = response.json()
    if isinstance(data, list):  # Se retornar uma lista com múltiplos registros
        output = []
        for record in data:
            output.append(f"Município: {record['municipio']} ({record['uf']}) - Categoria: {record['categoria']} | Subcategoria: {record['subcategoria']} | Quantidade: {record['quantidade']} | Situação: {record['situacao']}")
        return html.Div(output)
    else:  # Caso contrário, só um registro
        return f"Município: {data[0]['municipio']} ({data[0]['uf']}) - Categoria: {data[0]['categoria']} | Subcategoria: {data[0]['subcategoria']} | Quantidade: {data[0]['quantidade']} | Situação: {data[0]['situacao']}"

# Função para consultar as reclamações
@app.callback(
    Output("output-reclamacoes", "children"),
    [Input("categoria-dropdown", "value"),
     Input("subcategoria-dropdown", "value")]
)
def consultar_reclamacoes(categoria, subcategoria):
    url = f"http://localhost:3000/reclamacoes?categoria={categoria}&subcategoria={subcategoria}"
    response = requests.get(url)
    
    if response.status_code != 200:
        return "Erro ao buscar reclamações."
    
    data = response.json()
    if not data:
        return "Nenhuma reclamação encontrada para os filtros informados."
    
    output = []
    for record in data:
        output.append(f"Município: {record['municipio']} ({record['uf']}) - Categoria: {record['categoria']} | Subcategoria: {record['subcategoria']} | Quantidade: {record['quantidade']} | Situação: {record['situacao']}")
    return html.Div(output)

# Rodando o servidor Dash
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8050)


In [ ]:

!pip install flask flask-cors pandas openpyxl scipy --quiet


from flask import Flask, jsonify, request
from flask_cors import CORS
import pandas as pd
from scipy.stats import pearsonr


app = Flask(__name__)
CORS(app)

# Ler o arquivo Excel do índice de satisfação do consumidor
indice_satisfacao = "indice-satisfacao-consumidor.xlsx"
df_satisfacao = pd.read_excel(indice_satisfacao, nrows=5)

# Rota para consultar dados por geração de conjunto de dados http://localhost:3000/dados/45383
@app.route('/dados/<datGeracao>', methods=['GET'])
def consulta_dados_por_geracao(datGeracao):
    """
    Endpoint para consultar dados de geração de conjunto de dados por DatGeracaoConjuntoDados.
    """
    dados_geracao = df_satisfacao[df_satisfacao['DatGeracaoConjuntoDados'] == int(datGeracao)]
    if not dados_geracao.empty:
        return jsonify(dados_geracao.to_dict(orient='records'))
    else:
        return jsonify({"error": "Dados de geração não encontrados"}), 404

# Rota para consultar dados por agente http://localhost:3000/agente/Mux Energia
@app.route('/agente/<sigAgente>', methods=['GET'])
def consulta_agente(sigAgente):
    """
    Endpoint para consultar informações sobre um agente específico.
    """
    agente_data = df_satisfacao[df_satisfacao['SigAgente'].str.contains(sigAgente, case=False)]
    if not agente_data.empty:
        return jsonify(agente_data.to_dict(orient='records'))
    else:
        return jsonify({"error": "Agente não encontrado"}), 404

# Rota para consultar dados por ano http://localhost:3000/ano/2022
@app.route('/ano/<numAno>', methods=['GET'])
def consulta_por_ano(numAno):
    """
    Endpoint para consultar os dados por ano.
    """
    ano_data = df_satisfacao[df_satisfacao['NumAno'] == int(numAno)]
    if not ano_data.empty:
        return jsonify(ano_data.to_dict(orient='records'))
    else:
        return jsonify({"error": "Ano não encontrado"}), 404

# Rota para consultar a correlação entre qualidade, satisfação e confiança  http://localhost:3000/correlacao
@app.route('/correlacao', methods=['GET'])
def correlacao_indicadores():
    """
    Endpoint para calcular a correlação entre os indicadores de Qualidade, Satisfação e Confiança.
    """
    # Calcular a correlação entre os indicadores de Qualidade, Satisfação e Confiança
    qualidade = df_satisfacao['MdaIndicadorQualidade']
    satisfacao = df_satisfacao['MdaIndicadorSatisfacao']
    confianca = df_satisfacao['MdaIndicadorConfianca']

    # Remover valores nulos
    qualidade = qualidade.dropna()
    satisfacao = satisfacao.dropna()
    confianca = confianca.dropna()

    # Calcular as correlações de Pearson
    corr_qualidade_satisfacao, _ = pearsonr(qualidade, satisfacao)
    corr_qualidade_confianca, _ = pearsonr(qualidade, confianca)
    corr_satisfacao_confianca, _ = pearsonr(satisfacao, confianca)

    return jsonify({
        "correlacao_qualidade_satisfacao": corr_qualidade_satisfacao,
        "correlacao_qualidade_confianca": corr_qualidade_confianca,
        "correlacao_satisfacao_confianca": corr_satisfacao_confianca
    })

# Endpoint para retornar opções de agentes (para o Dropdown)
@app.route('/agentes', methods=['GET'])
def obter_agentes():
    """
    Endpoint para retornar todos os agentes disponíveis para o Dropdown.
    """
    agentes = df_satisfacao['SigAgente'].unique().tolist()
    return jsonify(agentes)

# Endpoint para retornar opções de anos (para o Dropdown)
@app.route('/anos', methods=['GET'])
def obter_anos():
    """
    Endpoint para retornar todos os anos disponíveis para o Dropdown.
    """
    anos = df_satisfacao['NumAno'].unique().tolist()
    return jsonify(anos)

from werkzeug.serving import run_simple

if __name__ == '__main__':
    run_simple('localhost', 3000, app, use_debugger=False, use_reloader=False)


In [ ]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import requests
import pandas as pd
import plotly.express as px

# Iniciando o app Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Função para pegar dados da API
def obter_dados(endpoint, parametro=''):
    url = f'http://localhost:3000/{endpoint}/{parametro}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Função para obter as opções de agentes e anos
def obter_opcoes_dropdown(endpoint):
    url = f'http://localhost:3000/{endpoint}'
    response = requests.get(url)
    if response.status_code == 200:
        return [{'label': item, 'value': item} for item in response.json()]
    else:
        return []

# Layout do Dashboard
app.layout = html.Div([
    dbc.Container([
        html.H1("Dashboard Interativo - Satisfação do Consumidor", className="text-center my-4"),

        # Dropdown para seleção de geração
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-geracao',
                options=[
                    {'label': '45383', 'value': '45383'},
                    # Adicione mais opções de geração conforme necessário
                ],
                value='45383',  # valor inicial
                placeholder="Selecione a Geração"
            ), width=4)
        ], className="mb-4"),

        # Gráfico de Geração
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-geracao'), width=12)
        ], className="mb-4"),

        # Dropdown para seleção de agente
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-agente',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione o Agente"
            ), width=4)
        ], className="mb-4"),

        # Gráfico de Agente
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-agente'), width=12)
        ], className="mb-4"),

        # Dropdown para seleção de ano
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-ano',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione o Ano"
            ), width=4)
        ], className="mb-4"),

        # Gráfico de Ano
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-ano'), width=12)
        ], className="mb-4"),

        # Gráfico de Correlação entre Indicadores
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-correlacao'), width=12)
        ])
    ])
])

# Populando os Dropdowns com opções da API (agentes e anos)
@app.callback(
    [Output('dropdown-agente', 'options'),
     Output('dropdown-ano', 'options')],
    Input('dropdown-geracao', 'value')
)
def atualizar_dropdowns(datGeracao):
    # Puxando as opções de agentes e anos
    agentes = obter_opcoes_dropdown('agentes')
    anos = obter_opcoes_dropdown('anos')
    return agentes, anos

# Callback para atualizar o gráfico de geração
@app.callback(
    Output('grafico-geracao', 'figure'),
    [Input('dropdown-geracao', 'value')]
)
def atualizar_grafico_geracao(datGeracao):
    dados_geracao = obter_dados('dados', datGeracao)
    if dados_geracao:
        df = pd.DataFrame(dados_geracao)
        fig = px.bar(df, x='SigAgente', y='MdaIndicadorSatisfacao', title=f"Qualidade vs Satisfação - Geração {datGeracao}")
        return fig
    return {}

# Callback para atualizar o gráfico de agente
@app.callback(
    Output('grafico-agente', 'figure'),
    [Input('dropdown-agente', 'value')]
)
def atualizar_grafico_agente(sigAgente):
    dados_agente = obter_dados('agente', sigAgente)
    if dados_agente:
        df = pd.DataFrame(dados_agente)
        fig = px.line(df, x='NumAno', y='MdaIndicadorQualidade', title=f"Qualidade por Ano - Agente {sigAgente}")
        return fig
    return {}

# Callback para atualizar o gráfico de ano
@app.callback(
    Output('grafico-ano', 'figure'),
    [Input('dropdown-ano', 'value')]
)
def atualizar_grafico_ano(numAno):
    dados_ano = obter_dados('ano', numAno)
    if dados_ano:
        df = pd.DataFrame(dados_ano)
        fig = px.bar(df, x='SigAgente', y='MdaIndicadorConfianca', title=f"Confiança por Agente - Ano {numAno}")
        return fig
    return {}

# Callback para atualizar o gráfico de correlação
@app.callback(
    Output('grafico-correlacao', 'figure'),
    [Input('dropdown-geracao', 'value')]
)
def atualizar_grafico_correlacao(datGeracao):
    dados_correlacao = obter_dados('correlacao', '')
    if dados_correlacao:
        correlacoes = {
            'Qualidade vs Satisfação': dados_correlacao['correlacao_qualidade_satisfacao'],
            'Qualidade vs Confiança': dados_correlacao['correlacao_qualidade_confianca'],
            'Satisfação vs Confiança': dados_correlacao['correlacao_satisfacao_confianca']
        }
        fig = px.bar(x=list(correlacoes.keys()), y=list(correlacoes.values()), title="Correlação entre Indicadores")
        fig.update_yaxes(range=[-1, 1])  # Ajustando a escala da correlação
        return fig
    return {}

# Rodando o servidor Dash
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)


In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import pandas as pd
from scipy.stats import pearsonr

# Criação da aplicação Flask
app = Flask(__name__)
CORS(app) 

df_interrupcoes = pd.read_excel("interrupcoes-energia-eletrica-2023.xlsx")

# Função para obter dados de interrupções de energia para uma unidade consumidora http://localhost:3000/interrupcoes/São Francisco
@app.route('/interrupcoes/<unidade_consumidora>', methods=['GET'])
def interrupcoes_por_unidade(unidade_consumidora):
    """
    Endpoint que retorna as interrupções de energia para uma unidade consumidora específica.
    """
    dados = df_interrupcoes[df_interrupcoes['DscConjuntoUnidadeConsumidora'].str.contains(unidade_consumidora, case=False)]
    return jsonify(dados.to_dict(orient='records'))

# Rota para consultar interrupções por ano http://localhost:3000/interrupcoes/ano/2023
@app.route('/interrupcoes/ano/<int:ano>', methods=['GET'])
def interrupcoes_por_ano(ano):
    """
    Endpoint que retorna as interrupções de energia elétrica para um determinado ano.
    """
    dados = df_interrupcoes[df_interrupcoes['NumAno'] == ano]
    return jsonify(dados.to_dict(orient='records'))

# Rota para consultar interrupções por Agente Regulador http://localhost:3000/interrupcoes/agente/ENERGISA
@app.route('/interrupcoes/agente/<agente>', methods=['GET'])
def interrupcoes_por_agente(agente):
    """
    Endpoint que retorna as interrupções de energia elétrica para um agente regulador específico.
    """
    dados = df_interrupcoes[df_interrupcoes['NomAgenteRegulado'].str.contains(agente, case=False)]
    return jsonify(dados.to_dict(orient='records'))

from werkzeug.serving import run_simple

if __name__ == '__main__':
    run_simple('localhost', 3000, app, use_debugger=False, use_reloader=False)


In [ ]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import requests
import plotly.express as px
import pandas as pd

# Criando o app Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Função para obter dados da API Flask
def obter_dados(endpoint, parametro=''):
    url = f'http://localhost:3000/{endpoint}/{parametro}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Layout do Dashboard
app.layout = html.Div([
    dbc.Container([
        html.H1("Dashboard Interativo - Interrupções de Energia", className="text-center my-4"),

        # Dropdown para seleção de unidade consumidora
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-unidade',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione a Unidade Consumidora"
            ), width=6)
        ], className="mb-4"),

        # Gráfico de Interrupções por Unidade Consumidora
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-unidade'), width=12)
        ], className="mb-4"),

        # Dropdown para seleção de ano
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-ano',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione o Ano"
            ), width=6)
        ], className="mb-4"),

        # Gráfico de Interrupções por Ano
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-ano'), width=12)
        ], className="mb-4"),

        # Dropdown para seleção de agente regulador
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-agente',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione o Agente Regulador"
            ), width=6)
        ], className="mb-4"),

        # Gráfico de Interrupções por Agente
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-agente'), width=12)
        ], className="mb-4"),
    ])
])

# Populando os Dropdowns com opções da API
@app.callback(
    [Output('dropdown-unidade', 'options'),
     Output('dropdown-ano', 'options'),
     Output('dropdown-agente', 'options')],
    Input('dropdown-unidade', 'value')
)
def atualizar_dropdowns(unidade):
    unidades = obter_dados('interrupcoes', 'unidades')
    anos = obter_dados('interrupcoes/ano', '')
    agentes = obter_dados('interrupcoes/agente', '')

    opcoes_unidade = [{'label': item['DscConjuntoUnidadeConsumidora'], 'value': item['DscConjuntoUnidadeConsumidora']} for item in unidades]
    opcoes_ano = [{'label': str(ano), 'value': ano} for ano in anos]
    opcoes_agente = [{'label': agente, 'value': agente} for agente in agentes]

    return opcoes_unidade, opcoes_ano, opcoes_agente

# Callback para atualizar o gráfico de Unidade Consumidora
@app.callback(
    Output('grafico-unidade', 'figure'),
    [Input('dropdown-unidade', 'value')]
)
def atualizar_grafico_unidade(unidade):
    if unidade:
        dados_unidade = obter_dados('interrupcoes', unidade)
        if dados_unidade:
            df = pd.DataFrame(dados_unidade)
            fig = px.line(df, x='NumMes', y='NumInterrupcao', title=f'Interrupções - {unidade}')
            return fig
    return {}

# Callback para atualizar o gráfico de Ano
@app.callback(
    Output('grafico-ano', 'figure'),
    [Input('dropdown-ano', 'value')]
)
def atualizar_grafico_ano(ano):
    if ano:
        dados_ano = obter_dados('interrupcoes/ano', ano)
        if dados_ano:
            df = pd.DataFrame(dados_ano)
            fig = px.bar(df, x='DscConjuntoUnidadeConsumidora', y='NumInterrupcao', title=f'Interrupções no Ano {ano}')
            return fig
    return {}

# Callback para atualizar o gráfico de Agente Regulador
@app.callback(
    Output('grafico-agente', 'figure'),
    [Input('dropdown-agente', 'value')]
)
def atualizar_grafico_agente(agente):
    if agente:
        dados_agente = obter_dados('interrupcoes/agente', agente)
        if dados_agente:
            df = pd.DataFrame(dados_agente)
            fig = px.line(df, x='NumMes', y='NumInterrupcao', title=f'Interrupções - Agente {agente}')
            return fig
    return {}

# Rodando o servidor Dash
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)


In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import pandas as pd
from scipy.stats import pearsonr

# Criação da aplicação Flask
app = Flask(__name__)
CORS(app)  # Permitir requisições de outros domínios (útil para testes)

# Carregar apenas as 5 primeiras linhas da tabela de reclamações
df_reclamacoes = pd.read_excel("reclamacoes-n1e2-distribuidoras-2023.xlsx", nrows=5)

# Verifique se os dados foram carregados corretamente
print(df_reclamacoes.head())

# Função para obter reclamações por agente regulador http://localhost:3000/reclamacoes/agente/Amazonas Energia
@app.route('/reclamacoes/agente/<agente>', methods=['GET'])
def reclamacoes_por_agente(agente):
    """
    Endpoint que retorna as reclamações para um agente regulador específico.
    """
    dados = df_reclamacoes[df_reclamacoes['SigAgente'].str.contains(agente, case=False)]
    return jsonify(dados.to_dict(orient='records'))

# Função para obter reclamações por município http://localhost:3000/reclamacoes/municipio/Manaus
@app.route('/reclamacoes/municipio/<municipio>', methods=['GET'])
def reclamacoes_por_municipio(municipio):
    """
    Endpoint que retorna as reclamações para um município específico.
    """
    dados = df_reclamacoes[df_reclamacoes['NomMunicipio'].str.contains(municipio, case=False)]
    return jsonify(dados.to_dict(orient='records'))

# Função para obter reclamações por tipo de reclamação http://localhost:3000/reclamacoes/tipo/122
@app.route('/reclamacoes/tipo/<int:tipo>', methods=['GET'])
def reclamacoes_por_tipo(tipo):
    """
    Endpoint que retorna as reclamações para um tipo específico.
    """
    dados = df_reclamacoes[df_reclamacoes['CodTipoReclamacao'] == tipo]
    return jsonify(dados.to_dict(orient='records'))

# Função para obter as reclamações por canal de contato http://localhost:3000/reclamacoes/canal/Telefônico
@app.route('/reclamacoes/canal/<canal>', methods=['GET'])
def reclamacoes_por_canal(canal):
    """
    Endpoint que retorna as reclamações para um canal de contato específico.
    """
    dados = df_reclamacoes[df_reclamacoes['NomCanalReclamacao'].str.contains(canal, case=False)]
    return jsonify(dados.to_dict(orient='records'))

# Executar a API no Jupyter Notebook
from werkzeug.serving import run_simple

if __name__ == '__main__':
    run_simple('localhost', 3000, app, use_debugger=False, use_reloader=False)


In [ ]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import requests
import plotly.express as px
import pandas as pd

# Criando o app Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Função para obter dados da API Flask
def obter_dados(endpoint, parametro=''):
    url = f'http://localhost:3000/{endpoint}/{parametro}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Layout do Dashboard
app.layout = html.Div([
    dbc.Container([
        html.H1("Dashboard Interativo - Reclamações", className="text-center my-4"),

        # Dropdown para seleção de agente regulador
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-agente',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione o Agente Regulador"
            ), width=6)
        ], className="mb-4"),

        # Gráfico de Reclamações por Agente
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-agente'), width=12)
        ], className="mb-4"),

        # Dropdown para seleção de município
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-municipio',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione o Município"
            ), width=6)
        ], className="mb-4"),

        # Gráfico de Reclamações por Município
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-municipio'), width=12)
        ], className="mb-4"),

        # Dropdown para seleção de tipo de reclamação
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-tipo',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione o Tipo de Reclamação"
            ), width=6)
        ], className="mb-4"),

        # Gráfico de Reclamações por Tipo
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-tipo'), width=12)
        ], className="mb-4"),

        # Dropdown para seleção de canal de contato
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dropdown-canal',
                options=[],
                value=None,  # valor inicial
                placeholder="Selecione o Canal de Reclamação"
            ), width=6)
        ], className="mb-4"),

        # Gráfico de Reclamações por Canal
        dbc.Row([
            dbc.Col(dcc.Graph(id='grafico-canal'), width=12)
        ], className="mb-4"),
    ])
])

# Populando os Dropdowns com opções da API
@app.callback(
    [Output('dropdown-agente', 'options'),
     Output('dropdown-municipio', 'options'),
     Output('dropdown-tipo', 'options'),
     Output('dropdown-canal', 'options')],
    Input('dropdown-agente', 'value')
)
def atualizar_dropdowns(agente):
    agentes = obter_dados('reclamacoes/agente', '')
    municipios = obter_dados('reclamacoes/municipio', '')
    tipos = obter_dados('reclamacoes/tipo', '')
    canais = obter_dados('reclamacoes/canal', '')

    opcoes_agente = [{'label': item['SigAgente'], 'value': item['SigAgente']} for item in agentes]
    opcoes_municipio = [{'label': item['NomMunicipio'], 'value': item['NomMunicipio']} for item in municipios]
    opcoes_tipo = [{'label': str(item['CodTipoReclamacao']), 'value': item['CodTipoReclamacao']} for item in tipos]
    opcoes_canal = [{'label': item['NomCanalReclamacao'], 'value': item['NomCanalReclamacao']} for item in canais]

    return opcoes_agente, opcoes_municipio, opcoes_tipo, opcoes_canal

# Callback para atualizar o gráfico de Reclamações por Agente Regulador
@app.callback(
    Output('grafico-agente', 'figure'),
    [Input('dropdown-agente', 'value')]
)
def atualizar_grafico_agente(agente):
    if agente:
        dados_agente = obter_dados('reclamacoes/agente', agente)
        if dados_agente:
            df = pd.DataFrame(dados_agente)
            fig = px.bar(df, x='NomMunicipio', y='QtdReclamacoes', title=f'Reclamações - Agente {agente}')
            return fig
    return {}

# Callback para atualizar o gráfico de Reclamações por Município
@app.callback(
    Output('grafico-municipio', 'figure'),
    [Input('dropdown-municipio', 'value')]
)
def atualizar_grafico_municipio(municipio):
    if municipio:
        dados_municipio = obter_dados('reclamacoes/municipio', municipio)
        if dados_municipio:
            df = pd.DataFrame(dados_municipio)
            fig = px.bar(df, x='SigAgente', y='QtdReclamacoes', title=f'Reclamações - Município {municipio}')
            return fig
    return {}

# Callback para atualizar o gráfico de Reclamações por Tipo
@app.callback(
    Output('grafico-tipo', 'figure'),
    [Input('dropdown-tipo', 'value')]
)
def atualizar_grafico_tipo(tipo):
    if tipo:
        dados_tipo = obter_dados('reclamacoes/tipo', tipo)
        if dados_tipo:
            df = pd.DataFrame(dados_tipo)
            fig = px.pie(df, names='NomMunicipio', values='QtdReclamacoes', title=f'Reclamações - Tipo {tipo}')
            return fig
    return {}

# Callback para atualizar o gráfico de Reclamações por Canal de Contato
@app.callback(
    Output('grafico-canal', 'figure'),
    [Input('dropdown-canal', 'value')]
)
def atualizar_grafico_canal(canal):
    if canal:
        dados_canal = obter_dados('reclamacoes/canal', canal)
        if dados_canal:
            df = pd.DataFrame(dados_canal)
            fig = px.bar(df, x='NomMunicipio', y='QtdReclamacoes', title=f'Reclamações - Canal {canal}')
            return fig
    return {}

# Rodando o servidor Dash
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
